## This is the official training script of MIL tasks
* Use google colab pro+ (high RAM+GPU)
* we use the P100 GPU for the Experiments

## The code and Training process along with all record are private
* Our github page: https://github.com/sagizty/MIL-SI
* The ROSE dataset is not publicly aviliable.
* However the MICCAI 2015 chanllenge dataset is avaliable for illustration.


In [1]:
# check GPU
!nvidia-smi

Mon Aug 15 10:18:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!date --date='+8 hour'  # CST time zone

Mon Aug 15 18:18:44 UTC 2022


## Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## create file-system enviroment
* mount your google drive first


In [ ]:
# create file-system enviroment
# mount the google drive first
# https://drive.google.com/drive/u/1/my-drive

# clear colab path
!rm -rf /data
!rm -rf /home/MIL_Experiment

# create path
!mkdir /home/MIL_Experiment
!mkdir /home/MIL_Experiment/runs
!mkdir /home/MIL_Experiment/code
!mkdir /home/MIL_Experiment/saved_models
!mkdir /home/MIL_Experiment/imaging_results

!mkdir /data
!mkdir /data/MIL_Experiment
!mkdir /data/MIL_Experiment/dataset

print('Folder Tree Creation completed!')

# get the latest code from Github MIL-SI official page
!git clone https://www.github.com/sagizty/MIL-SI.git /home/MIL_Experiment/code
print('code transfer from github completed!')

# copy runs if u want to compare
# !cp -r /content/drive/MyDrive/MIL_Experiment/runs/* /home/MIL_Experiment/runs
# print('tensorboard log transfer completed!')

# copy saved_models if u want to compare
# !cp -r /content/drive/MyDrive/MIL_Experiment/saved_models/* /home/MIL_Experiment/saved_models
# print('saved_models transfer completed!')

# get the MIL and CLS dataset from github
# by its zip
!cp /home/MIL_Experiment/code/sample_datasets/warwick_MIL.zip /data/MIL_Experiment/dataset/
!cp /home/MIL_Experiment/code/sample_datasets/warwick_CLS.zip /data/MIL_Experiment/dataset/
# unzip
!unzip -q /data/MIL_Experiment/dataset/warwick_MIL.zip -d /data/MIL_Experiment/dataset/
!unzip -q /data/MIL_Experiment/dataset/warwick_CLS.zip -d /data/MIL_Experiment/dataset/
# alter the path
!rm -rf /data/MIL_Experiment/dataset/warwick_MIL.zip
!rm -rf /data/MIL_Experiment/dataset/warwick_CLS.zip
print('data transfer completed!')

Folder Tree Creation completed!
Cloning into '/home/MIL_Experiment/code'...
remote: Enumerating objects: 841, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 841 (delta 18), reused 0 (delta 0), pack-reused 798
Receiving objects: 100% (841/841), 627.71 MiB | 29.71 MiB/s, done.
Resolving deltas: 100% (145/145), done.
Checking out files: 100% (622/622), done.
code transfer from github completed!
data transfer completed!


## Arrange the working enviorment

In [ ]:
# change working dir
import os
os.chdir("/home/MIL_Experiment/code")
!pwd

# get packages
!pip install -q torch==1.10.0+cu111 torchvision==0.11.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install tensorboardX
!pip install timm==0.5.4
!pip install notifyemail
!pip install ttach


/home/MIL_Experiment/code
     |████████████▌                   | 834.1 MB 69.0 MB/s eta 0:00:19tcmalloc: large alloc 1147494400 bytes == 0x390f6000 @  0x7f95db241615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |███████████████▉                | 1055.7 MB 1.4 MB/s eta 0:13:22tcmalloc: large alloc 1434370048 bytes == 0x7d74c000 @  0x7f95db241615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████            | 1336.2 MB 77.7 MB/s eta 0:00:11tcmalloc: large alloc 1792966656 bytes == 0x257e000 @  0x7f95db241615 0x592b76 0x

In [ ]:
!python --version

Python 3.7.13


In [ ]:
!pip list

Package                       Version
----------------------------- ----------------------------
absl-py                       1.2.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     2.1.9
autograd                      1.4
Babel                         2.10.3
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.

In [ ]:
!pip freeze>requirements.txt
!cp requirements.txt ../runs

## Start Training
* by command line
* use argparse to set down hyper-parameter

* 5-fold experiment is used here

# MIL Experiments

* Train

In [ ]:
!python MIL_train.py --model_idx ViT_384_401_PT_lf05_b4_p32_warwick_MIL --data_augmentation_mode 2 --edge_size 384 --batch_size 4 --patch_size 32 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_MIL --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

GPU: 0
*********************************setting*************************************
Namespace(CLS_MIL_head_weight=1.0, CLS_MIL_off=False, MIL_head_weight=1.0, Pre_Trained_model_path=None, backbone_PT_off=False, batch_size=4, check_minibatch=5, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_MIL', draw_root='/home/MIL_Experiment/runs', edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=0, intake_epochs=0, lr=1e-05, lrf=0.05, model_idx='ViT_384_401_PT_lf05_b4_p32_warwick_MIL', model_path='/home/MIL_Experiment/saved_models', num_epochs=50, num_workers=2, opt_name='Adam', paint=True, patch_size=32, shuffle_MIL_off=False)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 24, 24]         590,592
        PatchEmbed-2             [-1, 576, 768]             

* MIL Test

In [ ]:
!python MIL_test.py --model_idx ViT_384_401_PT_lf05_b4_p32_warwick_MIL --data_augmentation_mode 2 --edge_size 384 --batch_size 1 --patch_size 32  --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_MIL --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

*********************************setting*************************************
Namespace(CLS_MIL_head_weight=1.0, CLS_MIL_off=False, MIL_Stripe=False, MIL_head_weight=1.0, Pre_Trained_model_path=None, batch_size=1, check_minibatch=5, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_MIL', draw_root='/home/MIL_Experiment/runs', edge_size=384, enable_attention_check=False, enable_notify=False, enable_tensorboard=False, enable_visualize_check=False, gpu_idx=0, model_idx='ViT_384_401_PT_lf05_b4_p32_warwick_MIL', model_path='/home/MIL_Experiment/saved_models', paint=True, patch_size=32, shuffle_MIL_off=False, shuffle_attention_check=False, shuffle_dataloader=False)
GPU: 0
model loaded
model : ViT_384_401_PT_lf05_b4_p32_warwick_MIL
Epoch: Test
----------
Epoch: test     test index of 5 minibatch: 1      time used: 0.4074726104736328
minibatch AVG loss: 25.286715698242187
Epoch: test     test index of 5 minibatch: 2      time used: 0.2811000347137451
minibatch AVG loss: 

# Test & Imaging

Basic CLS CAM

In [ ]:
!python Test.py --model_idx ViT_384_401_PT_lf05_b4_p32_warwick_MIL --data_augmentation_mode 2 --MIL_Stripe --enable_attention_check --check_minibatch 5 --edge_size 384 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/imaging_results

class_names: ['benign', 'malignant']
model loaded
model : ViT_384_401_PT_lf05_b4_p32_warwick_MIL
*********************************setting*************************************
Namespace(MIL_Stripe=True, att_module='SimAM', attn_drop_rate=0.0, batch_size=1, check_minibatch=5, cls_token_off=False, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/imaging_results', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=True, enable_notify=False, enable_tensorboard=False, enable_visualize_check=False, gpu_idx=0, model_idx='ViT_384_401_PT_lf05_b4_p32_warwick_MIL', model_path='/home/MIL_Experiment/saved_models', num_classes=0, paint=True, pos_embedding_off=False)
Epoch: Test
----------
Epoch: test     test index of 5 minibatch: 1      time used: 0.2544424533843994
minibatch AVG loss: 0.002735888137249276
/home/MIL_Experiment/code/utils/visual_usage.py:18: UserWarning: Implicit dimension choice for softmax has been 

CAM on shuffled images (random batch)

In [ ]:
!python MIL_test.py --model_idx ViT_384_401_PT_lf05_b4_p32_warwick_MIL --data_augmentation_mode 2 --shuffle_attention_check --MIL_Stripe --edge_size 384 --shuffle_dataloader --batch_size 4 --patch_size 32 --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_MIL --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/imaging_results

*********************************setting*************************************
Namespace(CLS_MIL_head_weight=1.0, CLS_MIL_off=False, MIL_Stripe=True, MIL_head_weight=1.0, Pre_Trained_model_path=None, batch_size=4, check_minibatch=5, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_MIL', draw_root='/home/MIL_Experiment/imaging_results', edge_size=384, enable_attention_check=False, enable_notify=False, enable_tensorboard=False, enable_visualize_check=False, gpu_idx=0, model_idx='ViT_384_401_PT_lf05_b4_p32_warwick_MIL', model_path='/home/MIL_Experiment/saved_models', paint=True, patch_size=32, shuffle_MIL_off=False, shuffle_attention_check=True, shuffle_dataloader=True)
GPU: 0
model loaded
model : ViT_384_401_PT_lf05_b4_p32_warwick_MIL
Epoch: Test
----------
Epoch: test     test index of 5 minibatch: 1      time used: 0.7422976493835449
minibatch AVG loss: 0.0006218856753548608
/home/MIL_Experiment/code/utils/visual_usage.py:18: UserWarning: Implicit dimension choic

CAM on shuffled images (continuous batch)

In [ ]:
!python MIL_test.py --model_idx ViT_384_401_PT_lf05_b4_p32_warwick_MIL --data_augmentation_mode 2 --shuffle_attention_check --MIL_Stripe --edge_size 384 --batch_size 4 --patch_size 32 --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_MIL --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/imaging_results

*********************************setting*************************************
Namespace(CLS_MIL_head_weight=1.0, CLS_MIL_off=False, MIL_Stripe=True, MIL_head_weight=1.0, Pre_Trained_model_path=None, batch_size=4, check_minibatch=5, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_MIL', draw_root='/home/MIL_Experiment/imaging_results', edge_size=384, enable_attention_check=False, enable_notify=False, enable_tensorboard=False, enable_visualize_check=False, gpu_idx=0, model_idx='ViT_384_401_PT_lf05_b4_p32_warwick_MIL', model_path='/home/MIL_Experiment/saved_models', paint=True, patch_size=32, shuffle_MIL_off=False, shuffle_attention_check=True, shuffle_dataloader=False)
GPU: 0
model loaded
model : ViT_384_401_PT_lf05_b4_p32_warwick_MIL
Epoch: Test
----------
Epoch: test     test index of 5 minibatch: 1      time used: 0.7266407012939453
minibatch AVG loss: 0.0010575442924164236
/home/MIL_Experiment/code/utils/visual_usage.py:18: UserWarning: Implicit dimension choi

CAM on shuffled images (batch size = 1)

In [ ]:
!python MIL_test.py --model_idx ViT_384_401_PT_lf05_b4_p32_warwick_MIL --data_augmentation_mode 2 --shuffle_attention_check --MIL_Stripe --edge_size 384 --batch_size 1 --patch_size 32 --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_MIL --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/imaging_results

*********************************setting*************************************
Namespace(CLS_MIL_head_weight=1.0, CLS_MIL_off=False, MIL_Stripe=True, MIL_head_weight=1.0, Pre_Trained_model_path=None, batch_size=1, check_minibatch=5, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_MIL', draw_root='/home/MIL_Experiment/imaging_results', edge_size=384, enable_attention_check=False, enable_notify=False, enable_tensorboard=False, enable_visualize_check=False, gpu_idx=0, model_idx='ViT_384_401_PT_lf05_b4_p32_warwick_MIL', model_path='/home/MIL_Experiment/saved_models', paint=True, patch_size=32, shuffle_MIL_off=False, shuffle_attention_check=True, shuffle_dataloader=False)
GPU: 0
model loaded
model : ViT_384_401_PT_lf05_b4_p32_warwick_MIL
Epoch: Test
----------
Epoch: test     test index of 5 minibatch: 1      time used: 0.26824474334716797
minibatch AVG loss: 0.0035511413705535235
/home/MIL_Experiment/code/utils/visual_usage.py:18: UserWarning: Implicit dimension cho

# Synchronize files

In [ ]:
# change working dir
import os
os.chdir("/home/MIL_Experiment/code/utils")
!python check_log_json.py --draw_root /home/MIL_Experiment/runs --record_dir /home/MIL_Experiment/CSV_logs

In [ ]:
# create path on google drive
!mkdir /content/drive/MyDrive/MIL_SI_sample
# copy the results
!/bin/cp -rf /home/MIL_Experiment/* /content/drive/MyDrive/MIL_SI_sample/
print('results copy completed!')

mkdir: cannot create directory ‘/content/drive/MyDrive/MIL_SI_sample’: File exists
results copy completed!


In [ ]:
!date --date='+8 hour'  # CST time zone

Mon Aug 15 18:41:10 UTC 2022
